# Notebook: West Nile Virus Prediction for Chicago
---

## Data Gathering

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix as cm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
# data comes from Kaggle competition.  
train_data   = pd.read_csv('./assets/train.csv')
test_data    = pd.read_csv('./assets/test.csv', index_col='Id')
weather_data = pd.read_csv('./assets/weather_station_avg.csv')

In [3]:
train_data.shape, test_data.shape

((10506, 12), (116293, 10))

In [4]:
train_data.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [5]:
test_data.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
Id,,,,,,,,,,
1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [6]:
# train_data = pd.merge(train_data, weather_data, how='inner', on='Date', sort=False, validate='m:1')
train_data = train_data.merge(weather_data, on='Date')
test_data  = test_data.merge(weather_data, on="Date")

train_data.shape, test_data.shape

((10506, 39), (116293, 37))

## Data Processing and Engineering

In [7]:
# to sort the values of the bugs that can have west nile from the ones that don't.
def processing_species(df):
    for bug in df:
        if bug == 'CULEX PIPIENS/RESTUANS':
            return 'wnv_bug'
        elif bug == 'CULEX PIPIENS':
            return 'wnv_bug'
#         elif bug == 'CULEX RESTUANS':
#             return 'wnv_bug'
        elif bug == 'UNSPECIFIED CULEX':  # Assuming that the unspecified bug is a virus carrier.
            return 'wnv_bug'
        else:
            return 'no_wnv_bug'

train_data['Species'] = train_data[['Species']].apply(processing_species, axis=1)
test_data['Species']  = test_data[['Species']].apply(processing_species, axis=1)

In [8]:
# Break down to see if specific days are the most useful. My assumptions
def date_separate(df):
    df = df.copy()
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day'] = pd.DatetimeIndex(df['Date']).day
    return df

train_data = date_separate(train_data)
test_data  = date_separate(test_data)

In [9]:
# Removing date, because we 
def processing(df):
    df.drop(['Date', 
             'Address', 
             'Street', 
             'AddressNumberAndStreet', 
             'Unnamed: 0',
             ], axis=1, inplace=True)
    df['Trap'] = [x.strip('TABCabc') for x in df['Trap']]
    df['Trap'].astype(int)
    df = pd.get_dummies(df, columns=['Species'])
    return df

train_data = processing(train_data)
test_data  = processing(test_data)

In [10]:
train_data.head()

,Block,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Tmax,Tmin,Tavg,...,CodeSum_TS,CodeSum_TS BR,CodeSum_TS RA,CodeSum_TS RA BR,Daylight,Year,Month,Day,Species_no_wnv_bug,Species_wnv_bug
0,41,002,41.954690,-87.800991,9,1,0,88,60,74,...,0,0,0,0,896,2007,5,29,0,1
1,41,002,41.954690,-87.800991,9,1,0,88,60,74,...,0,0,0,0,896,2007,5,29,1,0
2,62,007,41.994991,-87.769279,9,1,0,88,60,74,...,0,0,0,0,896,2007,5,29,1,0
3,79,015,41.974089,-87.824812,8,1,0,88,60,74,...,0,0,0,0,896,2007,5,29,0,1
4,79,015,41.974089,-87.824812,8,4,0,88,60,74,...,0,0,0,0,896,2007,5,29,1,0


In [11]:
# the difference in shapes is fine because we have the Target still in the Training Data.
train_data.shape, test_data.shape

((10506, 38), (116293, 36))

In [12]:
target   = train_data.WnvPresent
features = train_data.drop(['WnvPresent', 'NumMosquitos'], axis=1) 

In [13]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42, stratify=target)

## Modeling

In [14]:
# This poor representation of data split between the two objectives makes it tough to create a valid
# model that won't just always say no.  We will need to bootstrap some data.
train_data.WnvPresent.value_counts(normalize=True)

0    0.947554
1    0.052446
Name: WnvPresent, dtype: float64

In [15]:
# using SMOTE to achieve a better ratio of wnv. SMOTE is a synthetic data bootstrapping.
# ratio 0.5 will set the data to be a 66% original data to 34% new data.
sm = SMOTE(random_state=42, ratio='auto')

X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

from collections import Counter

print(Counter(y_train_res).keys()) # 0 = Wnv not present, 1 = Wnv present.
print(Counter(y_train_res).values()) # The values represent the keys respectively.

dict_keys([1, 0])
dict_values([7466, 7466])


In [16]:
# Using these three different models for classification.
ss  = StandardScaler()

log = LogisticRegression()
rfc = RandomForestClassifier(n_estimators=14, random_state=42)
abc = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

In [17]:
# pipeline to streamline the process.
pipe_log = Pipeline([
    ('ss', ss),
    ('log', log)
])
pipe_rfc = Pipeline([
    ('ss', ss),
    ('rfc', rfc),
])
pipe_abc = Pipeline([
    ('ss', ss),
    ('abc', abc)
])

In [18]:
# These grid searches are used to find the best parameters. It is commented out right now
# because we found the best parameters already.
# grid_log = GridSearchCV(estimator=pipe_log,
#                         param_grid=params_log,
#                         scoring='roc_auc',
#                         cv=5)
# grid_rfc = GridSearchCV(estimator=pipe_rfc,
#                         param_grid=params_rfc,
#                         scoring='roc_auc',
#                         cv=5)
# grid_abc = GridSearchCV(estimator=pipe_abc,
#                         param_grid=params_abc,
#                         scoring='roc_auc',
#                         cv=5)

In [19]:
%%time
pipes    = [ 
    pipe_log, 
    pipe_rfc, 
    pipe_abc,
]
pipe_idx = {0: 'Logistic Regression', 
            1: 'Random Forest', 
            2: 'Adaboost'
           }

for idx, pipe in enumerate(pipes):
    pipe.fit(X_train_res, y_train_res)
    print('\nScore Train/Test: %s' % pipe_idx[idx])
    print(pipe.score(X_train, y_train))
    print(pipe.score(X_test, y_test))
#     print('Best params: %s' % pipe.best_params_)


Score Train/Test: Logistic Regression
0.7182383551212083
0.7285877426722497

Score Train/Test: Random Forest
0.9573549942886153
0.9196802436239055

Score Train/Test: Adaboost
0.747049117908364
0.7609440426341835
CPU times: user 6.95 s, sys: 241 ms, total: 7.19 s
Wall time: 4.74 s


In [20]:
feat = pipe_rfc.named_steps['rfc']

importance = feat.feature_importances_

# Daylight and months correlate to being the most important features.  Clearly based on time of year
# In addition with latitude and longitude to target the specific
sorted(list(zip(importance, features.keys())), reverse=True)

[(0.1026643305951347, 'Daylight'),
 (0.08789478808252225, 'Month'),
 (0.06699698448617455, 'Longitude'),
 (0.06533586401730876, 'Trap'),
 (0.062057463992374606, 'Species_wnv_bug'),
 (0.05899513741805331, 'Block'),
 (0.05651626988286098, 'Latitude'),
 (0.05391274139849504, 'Year'),
 (0.0527351941267659, 'Species_no_wnv_bug'),
 (0.04798729773853131, 'AddressAccuracy'),
 (0.038448020899463795, 'AvgSpeed'),
 (0.031689499359130575, 'ResultSpeed'),
 (0.02600363816661491, 'Cool'),
 (0.023213733471891006, 'Tavg'),
 (0.022934988331706374, 'Tmax'),
 (0.0217685058245409, 'Depart'),
 (0.020497783078221737, 'SeaLevel'),
 (0.02014542849830443, 'CodeSum_BR'),
 (0.01941930213352549, 'WetBulb'),
 (0.018865616628581713, 'StnPressure'),
 (0.017932324026749897, 'DewPoint'),
 (0.016735780743109243, 'Tmin'),
 (0.016407309457399245, 'ResultDir'),
 (0.016205450134349902, 'Day'),
 (0.01441614574582857, 'PrecipTotal'),
 (0.009815149740188489, 'CodeSum_RA'),
 (0.005585076003213741, 'CodeSum_RA BR'),
 (0.00175824

In [25]:
test_data.to_csv('test_data_transformed.csv', index=False)
train_data.to_csv('train_data_transformed.csv', index=False)

In [21]:
predictions = pipe_rfc.predict(test_data)

In [22]:
len(predictions)

116293

In [23]:
sample_submission = pd.read_csv('./assets/sampleSubmission.csv')
sample_submission['WnvPresent'] = predictions
# sample_submission.to_csv('submission_randomforest6.csv', index=False)

In [24]:
print(Counter(predictions).keys()) # 0 = Wnv not present, 1 = Wnv present.
print(Counter(predictions).values()) # The values represent the keys respectively.

dict_keys([0, 1])
dict_values([94107, 22186])


## Executive Summary

After thorough cleaning, engineering, and modeling we were able to accurately predict 70% of the time where the West Nile Virus would be present.  With higher prediction scores, we can reduce the cost of spraying to target the areas that will most likely have the virus.  The cost spraying for some areas of Chicago can range from \$300 per square mile to $20,000 per square mile.  With costs ranging this dramatically, having an informed decision about where to spray is paramount to managing a budget within the city while keeping the citizens healthy.  

Targeted sprays to handle the mosquito population will reduce environmental risks and healh risks attributed to spraying chemicals in a highly populated area.  It negatively affects bees, land mammals, and aquatic animals; which is a detriment to maintaining a healthy ecosystem.  

Moving forward it is important to find additonal features that affect where the virus is propogated.